In [8]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.models import HoverTool
from bokeh.io import push_notebook


#### helathcare.py functions

In [2]:
def import_data(data_path):
    df = pd.read_csv(data_path, header=0, encoding = "utf-8", na_values=["NA", "null", "", "NaN"])

    df = df.dropna(axis = 1, thresh=1)

    return df


def pivot_IHME(df):

    pivoted_df = df.pivot_table(index=['location', 'sex', 'cause', 'year'], 
                            columns='measure', 
                            values='val', 
                            aggfunc='first') 

    # Reset the index if needed
    pivoted_df = pivoted_df.reset_index()

    # Rename the columns for clarity
    pivoted_df = pivoted_df.rename(columns={'death': 'death_rate', 'incidence': 'incidence_rate'})

    pivoted_df.to_csv('data/pivotted_dataframe_IHME.csv', index=False)

    return pivoted_df


def pivot_WHO(df):

    pivoted_df = df.pivot_table(index=['ParentLocation', 'Location', 'Period'], 
                            columns='Indicator', 
                            values='Value', 
                            aggfunc='first') 

    # Reset the index if needed
    pivoted_df = pivoted_df.reset_index()

    pivoted_df.to_csv('data/pivotted_dataframe_WHO.csv', index=False)

    return pivoted_df

In [3]:

data_WHO = import_data("data/WHO data.csv")
data_WHO = data_WHO.drop(['ParentLocationCode', 'SpatialDimValueCode', 'IndicatorCode', 'Period type', 'Location type', 'ValueType', 'FactComments', 'FactValueNumeric', 'Language', 'IsLatestYear', 'DateModified'], axis=1)

print(data_WHO.info())


data_IHME_1 = import_data("data/IHME-1.csv")
data_IHME_2 = import_data("data/IHME-2.csv")
data_IHME_combined = pd.concat([data_IHME_1, data_IHME_2], axis=0, ignore_index=True)
data_IHME_combined = data_IHME_combined.drop(['age', 'metric', 'upper', 'lower'], axis=1)

print(data_IHME_combined.info())

df_IHME = pivot_IHME(data_IHME_combined)
df_WHO = pivot_WHO(data_WHO)

# graph_docs_by_country(df_WHO)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14189 entries, 0 to 14188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Indicator       14189 non-null  object
 1   ParentLocation  14189 non-null  object
 2   Location        14189 non-null  object
 3   Period          14189 non-null  int64 
 4   Value           14189 non-null  object
dtypes: int64(1), object(4)
memory usage: 554.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970632 entries, 0 to 970631
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   measure   970632 non-null  object 
 1   location  970632 non-null  object 
 2   sex       970632 non-null  object 
 3   cause     970632 non-null  object 
 4   year      970632 non-null  int64  
 5   val       970632 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 44.4+ MB
None


#### Plotly

In [4]:
def graph_docs_by_country_plotly(df):
    parent_locations = df['ParentLocation'].dropna().unique()
    parent_locations.sort()

    years = sorted(df['Period'].unique())

    region_dropdown = widgets.Dropdown(
        options=['All'] + list(parent_locations),
        value='All',
        description='Region:',
        style={'description_width': 'initial'}
    )

    year_dropdown = widgets.Dropdown(
        options=years,
        value=max(years),
        description='Year:',
        style={'description_width': 'initial'}
    )

    def update_plot(region, year):
        df_filtered = df[df['Period'] == year]

        if region != 'All':
            df_filtered = df_filtered[df_filtered['ParentLocation'] == region]

        df_filtered = df_filtered.sort_values(by='Medical doctors (per 10,000)', ascending=False)

        fig = px.bar(df_filtered, 
                     x='Location', 
                     y='Medical doctors (per 10,000)', 
                     title=f"Medical Doctors per 10,000 People ({year}) - {region}",
                     labels={'Location': 'Country'},
                     hover_data={'Medical doctors (per 10,000)': True, 'Location': True},
                     text_auto=True)

        fig.update_traces(marker_color='royalblue', hoverinfo="x+y", textposition='outside')

        fig.update_layout(
            xaxis_title="Country",
            yaxis_title="Doctors per 10,000 People",
            xaxis={'categoryorder': 'total descending'},
            hovermode="x unified",
            bargap=0.2,
            dragmode="zoom"
        )

        fig.show()

    interactive_plot = widgets.interactive(update_plot, region=region_dropdown, year=year_dropdown)
    display(region_dropdown, year_dropdown)
    update_plot(region_dropdown.value, year_dropdown.value)

graph_docs_by_country_plotly(df_WHO)


Dropdown(description='Region:', options=('All', 'Africa', 'Americas', 'Eastern Mediterranean', 'Europe', 'Sout…

Dropdown(description='Year:', index=69, options=(1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1…

#### Bokeh

In [9]:
output_notebook()


def graph_docs_by_country_bokeh(df):
    parent_locations = sorted(df['ParentLocation'].dropna().unique())
    years = sorted(df['Period'].unique())

    default_year = max(years)
    default_region = 'All'

    def get_filtered_data(region, year):
        df_filtered = df[df['Period'] == year]
        if region != 'All':
            df_filtered = df_filtered[df_filtered['ParentLocation'] == region]
        df_filtered = df_filtered.sort_values(by='Medical doctors (per 10,000)', ascending=False)
        return df_filtered

    df_initial = get_filtered_data(default_region, default_year)
    source = ColumnDataSource(df_initial)

    p = figure(
        x_range=list(df_initial['Location']), 
        height=500, 
        title=f"Medical Doctors per 10,000 People ({default_year}) - {default_region}",
        tools="pan,box_zoom,reset,save", 
        toolbar_location="above"
    )

    bars = p.vbar(x='Location', top='Medical doctors (per 10,000)', source=source, width=0.8, color="royalblue")

    p.xaxis.axis_label = "Country"
    p.yaxis.axis_label = "Doctors per 10,000 People"
    p.xgrid.grid_line_color = None
    p.y_range.start = 0

    hover = HoverTool()
    hover.tooltips = [("Country", "@Location"), ("Doctors per 10k", "@{Medical doctors (per 10,000)}")]
    p.add_tools(hover)

    region_select = Select(title="Region:", value=default_region, options=['All'] + parent_locations)
    year_select = Select(title="Year:", value=str(default_year), options=[str(y) for y in years])

    def update(attr, old, new):
        region = region_select.value
        year = int(year_select.value)
        df_updated = get_filtered_data(region, year)

        source.data = ColumnDataSource.from_df(df_updated)

        p.x_range.factors = list(df_updated['Location'])
        p.title.text = f"Medical Doctors per 10,000 People ({year}) - {region}"

        push_notebook()
        
    region_select.on_change('value', update)
    year_select.on_change('value', update)

    show(column(region_select, year_select, p), notebook_handle=True)


graph_docs_by_country_bokeh(df_WHO)


Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

